In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import keras

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load and Explore the Data
def load_and_preprocess_data(data_dir, label,  augmentation):
    data = []
    labels = []

    for filename in os.listdir(data_dir):
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (256, 256))
        img = img / 255.0
        data.append(img)
        labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    if augmentation:
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        datagen.fit(data)
        augmented_data = datagen.flow(data, labels, batch_size=len(data), shuffle=False).next()
        data, labels = augmented_data[0], labels

    return data, labels

In [4]:
resume_folder = '/content/drive/MyDrive/resumes'
non_resume_folder = '/content/drive/MyDrive/not_resumes'

In [5]:
# Load and preprocess data from the two folders
resume_data_aug, resume_labels_aug = load_and_preprocess_data(resume_folder, 1, True)
non_resume_data_aug, non_resume_labels_aug = load_and_preprocess_data(non_resume_folder, 0, True)

resume_data, resume_labels = load_and_preprocess_data(resume_folder, 1, False)
non_resume_data, non_resume_labels = load_and_preprocess_data(non_resume_folder, 0, False)

In [6]:
# Combine data and labels
X_train_aug = np.concatenate((resume_data_aug, non_resume_data_aug), axis=0)
y_train_aug = np.concatenate((resume_labels_aug, non_resume_labels_aug), axis=0)

X_train = np.concatenate((resume_data, non_resume_data), axis=0)
y_train = np.concatenate((resume_labels, non_resume_labels), axis=0)

In [7]:
# Shuffle the data
augmented_data = list(zip(X_train_aug, y_train_aug))
np.random.shuffle(augmented_data)
X_train_aug, y_train_aug = zip(*augmented_data)
X_train_aug = np.array(X_train_aug)
y_train_aug = np.array(y_train_aug)

y_train_aug = to_categorical(y_train_aug, num_classes=2)
y_train = to_categorical(y_train, num_classes=2)

X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
    X_train_aug, y_train_aug, test_size=0.3, random_state=42
)

In [8]:
# build the model
model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(256,256,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.8),
    keras.layers.Dense(2,activation='softmax')
])

In [9]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 128)       46592     
                                                                 
 batch_normalization (Batch  (None, 62, 62, 128)       512       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 128)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 256)       819456    
                                                                 
 batch_normalization_1 (Bat  (None, 31, 31, 256)       1024      
 chNormalization)                                                
                                                        

In [10]:

print("X_train_split shape:", X_train_split.shape)
print("y_train_split shape:", y_train_split.shape)

X_train_split shape: (70, 256, 256, 3)
y_train_split shape: (70, 2)


In [13]:
# Train the model
history = model.fit(X_train_split, y_train_split, epochs=5, validation_data=(X_test_split, y_test_split))

Epoch 1/5
3/3 [==============================] - 18s 5s/step - loss: 1.9187 - accuracy: 0.7714 - val_loss: 5.7803 - val_accuracy: 0.4667
Epoch 2/5
3/3 [==============================] - 17s 4s/step - loss: 0.7302 - accuracy: 0.8571 - val_loss: 16.6467 - val_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 13s 3s/step - loss: 0.8295 - accuracy: 0.8429 - val_loss: 20.2421 - val_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 13s 4s/step - loss: 0.5252 - accuracy: 0.8714 - val_loss: 11.5685 - val_accuracy: 0.4000
Epoch 5/5
3/3 [==============================] - 11s 3s/step - loss: 1.0549 - accuracy: 0.7714 - val_loss: 15.2309 - val_accuracy: 0.4333


In [14]:
loss, accuracy = model.evaluate(X_test_split, y_test_split)
test_accuracy_percentage = accuracy * 100
print(f'Test Accuracy: {test_accuracy_percentage:.2f}%')

1/1 [==============================] - 1s 1s/step - loss: 15.2309 - accuracy: 0.4333
Test Accuracy: 43.33%
